In [ ]:
# Transform UserID and MovieID into sequential indices
user_encoder = {user: idx for idx, user in enumerate(rating_df['userId'].unique())}
movie_encoder = {movie: idx for idx, movie in enumerate(rating_df['movieId'].unique())}

rating_df['userId'] = rating_df['userId'].map(user_encoder)
rating_df['movieId'] = rating_df['movieId'].map(movie_encoder)

num_users = len(user_encoder)
num_movies = len(movie_encoder)

In [ ]:
# Create Edge for Graph
# We generate edge between user and movie when user rates movie higher than (or equal to) 1
# Adjacency matrix 대신 edge_index를 넣어서도 진행가능하다.
def create_edge_index(df, rating_threshold=1.0):
    src, dst = [], []
    for _, row in df.iterrows():
        if row['rating'] >= rating_threshold:
            src.append(row['userId'])
            # item indices after user indices
            dst.append(row['movieId'] + num_users)
    return torch.tensor([src, dst], dtype=torch.long)

edge_index = create_edge_index(rating_df)
print(edge_index)

In [ ]:
# Split indices into train/val/test set (label for train/val/test set)
train_indices, test_indices = train_test_split(range(edge_index.size(1)), test_size=0.2)
# edge_index의 shape은 (2, num_edges), 따라서 edge_index.size(1) = num_edges (간선 개수)
val_indices, test_indices = train_test_split(test_indices, test_size=0.5)

train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]
# edge에서는 다른 분야와 다른게 index 형태로 접근하는 것을 체크해 두기


In [ ]:
class NGCFLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout=0.1):
        super().__init__()
        self.W1 = nn.Linear(input_dim, output_dim)
        self.W2 = nn.Linear(input_dim, output_dim)
            # self.dropout = nn.Dropout(dropout)
        self.leaky_relu = nn.LeakyReLU(0.2)

    def forward(self, edge_index, node_features, user_num, item_num):
        '''
        edge_index : 엣지 정보 (src, dst)의 집합.
        node_features: node별 이전 layer에서 생성된 벡터 정보가 담긴 matrix (H^(l-1)) (|V| * d)
        '''

        src, dst = edge_index # src : user, dst : movie

            # calculate node degree
        deg = torch.zeros(node_features.size(0), device=node_features.device)
            # calculate user degree
        deg.index_add_(0, src, torch.ones_like(src, dtype=torch.float))
            # calculate movie degree
        deg.index_add_(0, dst, torch.ones_like(dst, dtype=torch.float))

            # calculate 1/(root(deg(u)) * root(deg(i))) for all edge
        norm = 1.0/torch.sqrt(deg[src]*deg[dst])

        src_feat = node_features[src] # H_u
        dst_feat = node_features[dst] # H_i

            # edge_messages for user(src) = m_(u<-i)) 결과 저장.
            # Hint: step1. self.W1(h_i) + self.W2(h_u * h_i) 계산
            # Hint: step2. 최종 m_(u<-i)를 위해선 앞선 norm을 앞서 계산한 message에 곱하기
        edge_messages_for_src = self.W1(dst_feat) + self.W2(dst_feat*src_feat)
        edge_messages_for_src *= norm.unsqueeze(1)  # unsqueeze(1)는 shape 맞추기
            #단순히 neighbor feature만 쓰지 않고, 두 노드의 상호작용을 함께 학습
            #neighbor 정보 + interaction 정보 → 최종 edge 메시지
            # W1에는 1개짜리, 상대방(src <->dst)이 들어가거 W2에는 2개 다 들어간다고 생각하자.(순서는 역시 상대방부터)

            # edge_messages for movie(dst) = m_(i<-u)) 결과 저장.
            # Hint: step1. self.W1(h_u) + self.W2(h_i * h_u) 계산
            # Hint: step2. 최종 m_(i<-u)를 위해선 앞선 norm을 앞서 계산한 message에 곱하기
        edge_messages_for_dst = self.W1(src_feat) + self.W2(src_feat*dst_feat)
        edge_messages_for_dst *= norm.unsqueeze(1)

            # aggregated_features = Combine()의 결과 저장.
        aggregated_messages = torch.zeros_like(node_features)
            # m_(u<-u) = self.W1(h_u) 계산해 더해주기
        aggregated_messages.index_add(0, src, edge_messages_for_src)
        aggregated_messages[:user_num] += self.W1(node_features[:user_num])

            # m_(i<-i) = self.W1(h_i) 계산해 더해주기
        aggregated_messages.index_add_(0, dst, edge_messages_for_dst)
        aggregated_messages[user_num:] += self.W1(node_features[user_num:])
        # []안의 인덱스를 잘 봐야 할 듯. src는 user 관련된거니까 0~user_num-1 까지 : [:user_num], dst는 그 뒤니까 user_num: 로 

        aggregated_features = self.leaky_relu(aggregated_messages)

        # engineering approach
        # aggregated_features = self.dropout(aggregated_features)
        return aggregated_features

In [ ]:
### TODO: NGCF 모델 완성.

class NGCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, layer_dims, dropout=0.1):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.node_embeddings = nn.Embedding(self.num_users+self.num_items,self.embedding_dim)
        nn.init.xavier_uniform_(self.node_embeddings.weight) # 일단 외워라...

        self.layers = nn.ModuleList([
            NGCFLayer(input_dim=(embedding_dim if i == 0 else layer_dims[i - 1]),
                      output_dim=layer_dims[i], dropout=dropout)
            for i in range(len(layer_dims))
        ])

    def forward(self, edge_index):
        node_features = self.node_embeddings.weight # (num_nodes, d₀),  d₀는 초기 임베딩 차원
        layer_outputs = [node_features]
        for layer in self.layers:
            node_features = layer(edge_index, node_features,self.num_users, self.num_items)
            layer_outputs.append(node_features)

        # Hint: NGCF의 final feature(representation)은 layer 별 feature에 대한 concatenated vector
        # Hint: 최종 final feature matrix에는 [feuture_vector for users + feature_vector for items]가 들어있음.

        final_features = torch.concat(layer_outputs,dim=-1) # 레이어별 임베딩을 feature dimension 기준으로 이어붙임(concatenate)
        user_features = final_features[:self.num_users] # self.num_users 를 기억해보자.
        item_features = final_features[self.num_users:]
        return user_features, item_features

    def bpr_loss(self, user_emb, pos_item_emb, neg_item_emb, reg_weight=1e-4):
        pos_scores = torch.sum(user_emb * pos_item_emb, dim=1)
        neg_scores = torch.sum(user_emb * neg_item_emb, dim=1)
        loss = -torch.mean(F.logsigmoid(pos_scores - neg_scores)) #평균을 취하고 음수 부호를 붙여서 최대화 문제를 최소화 문제로 변환
        reg_loss = reg_weight * (user_emb.norm(2).pow(2) + pos_item_emb.norm(2).pow(2) \ #각 임베딩의 L2 norm 제곱을 더함 → 파라미터 크기 제한
                                 + neg_item_emb.norm(2).pow(2)) / user_emb.size(0) # 배치 크기 기준으로 평균화
        return loss + reg_loss

In [ ]:
### 위 코드에서 layer_outputs와 final_features에 대한 보충 설명 예시
    num_users = 3, num_items = 2 → num_nodes = 5
        초기 임베딩 차원 d₀ = 4
        레이어 1 출력 차원 d₁ = 8
        레이어 2 출력 차원 d₂ = 8
    layer_outputs:
        H^(0): (5, 4)
        H^(1): (5, 8)
        H^(2): (5, 8)
    final_features: (5, 4+8+8) = (5, 20)
    user_features: (3, 20)
    item_features: (2, 20)

In [ ]:
def evaluate(user_features, item_features, test_edge_index, k):
    # user_features : (num_users, d)
    # item_features : (num_items, d)
    # test_edge_index: shape (2, E_test) (테스트 간선 집합)
    user_pos_items = defaultdict(list) # 존재하지 않는 키를 호출하면 자동으로 빈 리스트([])를 생성해준다고...
    E_test = test_edge_index.size(1) # test_edge_index가 (2, 1000)이면 E_test = 1000 
                                    # → 테스트 데이터에 1000개의 사용자-아이템 상호작용이 있다는 뜻
    for i in range(E_test):
        u = test_edge_index[0, i].item() # 결과적으로 u는 사용자 ID (0 ~ num_users-1 범위).
        it = test_edge_index[1, i].item() - num_users #  it는 아이템 ID (0 ~ num_items-1 범위).
        user_pos_items[u].append(it) # 예: user_pos_items[3] = [10, 25, 47] 
                                # → 사용자 3은 아이템 10, 25, 47을 긍정적으로 상호작용한 기록이 있음.

    recalls, precisions, ndcgs = [], [], []
    for user, pos_items in user_pos_items.items(): 
        user_emb = user_features[user] # user_features[user]는 특정 사용자 user의 임베딩 벡터 (d,)를 가져옴
        scores = torch.matmul(item_features, user_emb) # item_features는 (num_items, d), 결과는 (num_items,)
        topk_scores, topk_indices = torch.topk(scores, k=k) # 용자에게 추천할 Top‑K 아이템 후보군을 뽑아냅
        topk_indices = topk_indices.cpu().numpy().tolist() # 이후 조건문에서 리스트 연산을 쉽게 하기 위해 변환하는 과정

        hits = 0
        dcg = 0.0
        idcg = 0.0
        n_pos = len(pos_items)

        for rank, item_idx in enumerate(topk_indices):
            if item_idx in pos_items:
                hits += 1
                dcg += 1.0 / math.log2(rank + 2) # rank는 0부터 시작하므로 rank+2
                    # 예: 1등(rank=0) → 1/log2(2) = 1.0 / 2등(rank=1) → 1/log2(3) ≈ 0.63
        for rank in range(min(n_pos, k)):
            idcg += 1.0 / math.log2(rank + 2)

        recall_u = hits / n_pos
        precision_u = hits / k
        ndcg_u = dcg / idcg if idcg > 0 else 0.0

        recalls.append(recall_u)
        precisions.append(precision_u)
        ndcgs.append(ndcg_u)

    recall = np.mean(recalls)
    precision = np.mean(precisions)
    ndcg = np.mean(ndcgs)
    return recall, precision, ndcg

In [ ]:
def train(model, optimizer, train_edge_index, val_edge_index, num_epochs, batch_size, device, k):
    model.to(device)
    train_edge_index = train_edge_index.to(device)
    val_edge_index = val_edge_index.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = len(train_edge_index[0]) // batch_size

        for _ in range(num_batches):
            indices = torch.randint(0, train_edge_index.size(1), (batch_size,), device=device)
                # low = 0, high = train_edge_index.size(1) → 간선 개수
                # size = (batch_size,) → 배치 크기만큼 인덱스를 뽑음, device = device → CPU/GPU 위치 지정
            user_indices = train_edge_index[0, indices] # 랜덤하게 선택된 간선들의 사용자 노드 인덱스 집합
            pos_item_indices = train_edge_index[1, indices] - num_users 
                # 현재 배치에서 학습에 사용할 긍정 아이템 인덱스(0 ~ num_items-1 범위)
            neg_item_indices = torch.randint(0, num_movies, (batch_size,), device=device)
                # 아무 영화나 뽑아와서 '보지않은'영화로 간주하고 negative sample로 쓰자.
                # 보지 않은 영화임을 보장할 수 없는데 단순 실습이라서 그런듯.

            user_features, item_features = model(train_edge_index)
            u_emb = user_features[user_indices]
            pos_emb = item_features[pos_item_indices]
            neg_emb = item_features[neg_item_indices]

            loss = model.bpr_loss(u_emb, pos_emb, neg_emb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / num_batches:.4f}")

        if (epoch + 1) % 5 == 0:
            model.eval()
            with torch.no_grad():
                user_features, item_features = model(train_edge_index)
                recall, precision, ndcg = evaluate(user_features.cpu(), item_features.cpu(), val_edge_index, k)
                print(f"[Validation] Epoch {epoch + 1}: Recall@{k}: {recall:.4f}, Precision@{k}: {precision:.4f}, NDCG@{k}: {ndcg:.4f}")

In [ ]:
def test(model, train_edge_index, test_edge_index, k, device):
    model.eval()
    train_edge_index = train_edge_index.to(device)
    test_edge_index = test_edge_index.to(device)

    with torch.no_grad():
        user_features, item_features = model(train_edge_index)

    user_features = user_features.cpu()
    item_features = item_features.cpu()

    recall, precision, ndcg = evaluate(user_features, item_features, test_edge_index, k)
    recall = round(recall, 4)
    precision = round(precision, 4)
    ndcg = round(ndcg, 4)

    print(f"Recall@{k}: {recall:.4f}, Precision@{k}: {precision:.4f}, NDCG@{k}: {ndcg:.4f}")
    return recall, precision, ndcg